# Start coding and Enjoy the journey

## Install the repo

Details on modified/added files with respect to the original repo can be found in Visual-Place-Recognition-Project\modifiedFiles.md

In [ ]:
!git clone --recursive https://github.com/AML-Project-2025-2026/Visual-Place-Recognition-Project.git

## Install dependencies

In [ ]:
%cd Visual-Place-Recognition-Project/image-matching-models
!pip install -e .[all]

In [ ]:
!pip install faiss-cpu

## Download Datasets

In [ ]:
!python download_datasets.py

## Run your First VPR Evalutation



In [ ]:
!python VPR-methods-evaluation/main.py \
--num_workers 8 \
--batch_size 32 \
--log_dir log_dir \
--method=mixvpr --backbone=ResNet50 --descriptors_dimension=4096 \
--image_size 512 512 \
--database_folder 'data/sf_xs/val/database' \
--queries_folder 'data/sf_xs/val/queries' \
--num_preds_to_save 20 \
--recall_values 1 5 10 20 \
--save_for_uncertainty

## Run Image Matching on Retrieval Results



In [ ]:
!python match_queries_preds.py \
--preds-dir 'logs/log_dir/2025-12-21_09-01-13/preds' \
--out-dir 'logs/log_dir/2025-12-21_09-01-13/inliers_loftr' \
--matcher 'loftr' \
--device 'cuda' \
--num-preds 20

In [ ]:
#Run this cell to obtain the average processing time per query (not including pure reranking)

import torch
from pathlib import Path
import numpy as np

#Change here the path of the folder
folder = Path("logs/log_dir/2025-12-29_15-05-42/inliers_loftr")
files = sorted(folder.glob("*.torch"))

if not files:
    raise ValueError(f"Nessun file .torch trovato in {folder}")

all_query_times = []

for f in files:
    try:
        # weights_only=False to load "timing" dictionaries
        data = torch.load(f, weights_only=False, map_location='cpu')
        
        if "timing" in data and "total_query_time" in data["timing"]:
            all_query_times.append(data["timing"]["total_query_time"])
        else:
            print(f"{f.name} non contiene info sul timing.")
    except Exception as e:
        print(f"Errore nel caricamento di {f.name}: {e}")

if all_query_times:
    avg_query_time = np.mean(all_query_times)
    std_query_time = np.std(all_query_times)
    print(f"Risultati su {len(all_query_times)} query")
    print(f"Tempo medio per query: {avg_query_time:.4f} s")
    print(f"Deviazione standard: {std_query_time:.4f} s")
    print(f"Tempo medio stimato per singolo match: {avg_query_time/20:.4f} s")
else:
    print("Nessun dato di timing valido estratto.")

## Check Re-ranking Performance

In [ ]:
!python reranking.py \
--preds-dir 'logs/log_dir/2025-12-29_15-05-42/preds' \
--inliers-dir 'logs/log_dir/2025-12-29_15-05-42/inliers_loftr' \
--num-preds 20 \
--recall-values 1 5 10 20

In [ ]:
#Run this cell to draw the plots for the correlation between correctness and number of inliers (Top-1 retrieval)"

import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
import os
from pathlib import Path
from util import get_list_distances_from_preds

#Change here the paths of the folders
preds_folder = 'logs/log_dir/2025-12-28_13-41-54/preds'
inliers_folder = Path('logs/log_dir/2025-12-28_13-41-54/inliers_loftr')
threshold = 25

inliers_correct = []
inliers_wrong = []

txt_files = glob(os.path.join(preds_folder, "*.txt"))
txt_files.sort(key=lambda x: int(Path(x).stem))

print(f"Analizzando {len(txt_files)} query")

for txt_file_query in tqdm(txt_files):
    #get_list_distances_from_preds returns the distances in txt file order
    distances = get_list_distances_from_preds(txt_file_query)
    dist_top1 = distances[0] # Because interested only in the first one
    
    #Retrieve the inliers of the first prediction
    torch_file_query = inliers_folder.joinpath(Path(txt_file_query).name.replace('txt', 'torch'))
    query_data = torch.load(torch_file_query, weights_only=False)
    #results[0] matches the first line of the txt file
    inliers_top1 = query_data['results'][0]['num_inliers'] 
    
    if dist_top1 <= threshold:
        inliers_correct.append(inliers_top1)
    else:
        inliers_wrong.append(inliers_top1)

#plot 

plt.figure(figsize=(10, 6))

plt.hist(inliers_wrong, bins=25, alpha=0.6, label='Wrong queries (dist > 25m)', color='red', density=True)

plt.hist(inliers_correct, bins=25, alpha=0.6, label='Correct queries (dist <= 25m)', color='green', density=True)

plt.title("Correlation between correctness and number of inliers (Top-1 retrieval)")
plt.xlabel("Number of inliers")
plt.ylabel("Density of  queries")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
#Run this to obtain the best threshold for adaptive reranking (smallest threshold that keeps 99% of performance). Note that --avg-full-rerank-time is passed as an argument to be changed
!python adaptive_reranking.py --preds-dir 'logs/log_dir/2025-12-28_13-41-54/preds' inliers-dir 'logs/log_dir/2025-12-28_13-41-54/inliers_loftr' --taus 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 --num-preds 20 --avg-full-rerank-time 3.2708


In [ ]:
#Run this to evaluate the performance and savings for adaptive reranking with a specific threshold. Note that --avg-full-rerank-time is passed as an argument to be changed
!python adaptive_reranking_eval.py --preds-dir 'logs/log_dir/2025-12-28_13-41-54/preds' --inliers-dir 'logs/log_dir/2025-12-28_13-41-54/inliers_loftr' --tau 20 --avg-full-rerank-time 3.2708

## Perform Uncertainty Evalutation

In [ ]:
!python -m vpr_uncertainty.eval \
--preds-dir '<path-to-predictions-folder>' \
--inliers-dir '<path-to-inliers-folder>' \
--z-data-path '<path-to-z-data-file>'